In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [21]:
# Necessary imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score

# Data Loading
data = pd.read_csv('/kaggle/input/apple-watch-and-fitbit-data/aw_fb_data.csv')
df = data.drop(['device', 'Unnamed: 0', 'X1', 'age', 'gender', 'entropy_heart', 'entropy_setps',
                'corr_heart_steps', 'norm_heart', 'intensity_karvonen',
                'sd_norm_heart', 'steps_times_distance', 'hear_rate'], axis=1)

# Label Encoding
label_encoder = LabelEncoder()
le = label_encoder.fit(df['activity'])
ac_data = le.transform(df['activity'])
df['activity'] = ac_data

# Train-Test Split
target = df['activity']
feature = df.drop(columns='activity')
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=0)

# Data Preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression Model with Scaling and Cross-Validation
lr_model_scaled = LogisticRegressionCV(Cs=10, cv=5, penalty='l2', solver='lbfgs', max_iter=1000, random_state=0)
lr_model_scaled.fit(X_train_scaled, y_train)

# Predictions and Evaluation
y_pred_lr_scaled = lr_model_scaled.predict(X_test_scaled)
cm_lr_scaled = confusion_matrix(y_test, y_pred_lr_scaled)
accuracy_lr_scaled = accuracy_score(y_test, y_pred_lr_scaled)

# Display Confusion Matrix and Accuracy
print('Confusion Matrix = \n', cm_lr_scaled)
print("Accuracy of Logistic Regression Model (with Scaling) = ", accuracy_lr_scaled)


Confusion Matrix = 
 [[233   6  11  33   0   0]
 [100   7  42  18   0   0]
 [104  10  56  22   1   0]
 [179  15  21  30   1   0]
 [111   8  50  17   0   0]
 [126   5  15  31   1   0]]
Accuracy of Logistic Regression Model (with Scaling) =  0.2601755786113328


In [25]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Load the data
data = pd.read_csv('/kaggle/input/apple-watch-and-fitbit-data/aw_fb_data.csv')
df = data.drop(['device', 'Unnamed: 0', 'X1', 'age', 'gender', 'entropy_heart', 'entropy_setps',
                'corr_heart_steps', 'norm_heart', 'intensity_karvonen',
                'sd_norm_heart', 'steps_times_distance', 'hear_rate'], axis=1)

# Label Encoding
label_encoder = LabelEncoder()
le = label_encoder.fit(df['activity'])
df['activity'] = le.transform(df['activity'])

# Train-Test Split
target = df['activity']
feature = df.drop(columns='activity')
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=0)

# Data Preprocessing
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define the TensorFlow model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(len(np.unique(target)), activation='softmax')  # Output layer for classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=20, batch_size=32, validation_data=(X_test_scaled, y_test))

# Evaluate the model
y_pred_tf = np.argmax(model.predict(X_test_scaled), axis=-1)
cm_tf = confusion_matrix(y_test, y_pred_tf)
accuracy_tf = accuracy_score(y_test, y_pred_tf)

# Display Confusion Matrix and Accuracy
print('Confusion Matrix (TensorFlow) = \n', cm_tf)
print("Accuracy of TensorFlow Model = ", accuracy_tf)
threshold = 0.8
if accuracy_tf < threshold:
    print("Your activity prediction accuracy is below 80%. Consider incorporating more physical activities.")
else:
    print("Congratulations! Your activity prediction accuracy is above 80%. Keep up the good work!")

Epoch 1/20
157/157 [==============================] - 1s 3ms/step - loss: 1.7288 - accuracy: 0.2590 - val_loss: 1.7039 - val_accuracy: 0.2785
Epoch 2/20
157/157 [==============================] - 0s 2ms/step - loss: 1.6861 - accuracy: 0.2814 - val_loss: 1.6849 - val_accuracy: 0.2761
Epoch 3/20
157/157 [==============================] - 0s 2ms/step - loss: 1.6675 - accuracy: 0.2824 - val_loss: 1.6652 - val_accuracy: 0.2921
Epoch 4/20
157/157 [==============================] - 0s 2ms/step - loss: 1.6537 - accuracy: 0.2940 - val_loss: 1.6565 - val_accuracy: 0.3001
Epoch 5/20
157/157 [==============================] - 0s 2ms/step - loss: 1.6355 - accuracy: 0.3161 - val_loss: 1.6587 - val_accuracy: 0.2881
Epoch 6/20
157/157 [==============================] - 0s 2ms/step - loss: 1.6249 - accuracy: 0.3187 - val_loss: 1.6455 - val_accuracy: 0.3041
Epoch 7/20
157/157 [==============================] - 0s 2ms/step - loss: 1.6135 - accuracy: 0.3203 - val_loss: 1.6292 - val_accuracy: 0.3144
Epoch 

In [26]:
pip install Flask


Note: you may need to restart the kernel to use updated packages.
